In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# %% Read data
df1 = pd.read_excel("/content/Data - FF.xlsx", sheet_name="Price")
df1.to_csv("Price.csv")
df2 = pd.read_excel("/content/Data - FF.xlsx", sheet_name="Bookvalue")
df2.to_csv("Bookvalue.csv")
df3 = pd.read_excel("/content/Data - FF.xlsx", sheet_name="Marketvalue")
df3.to_csv("Marketvalue.csv")

In [7]:
# %% Price_day7
price = pd.read_csv('Price.csv', low_memory=False, index_col=1).iloc[:,1:]
# 449

# Drop 'ETF'
def ETF(df):
  l = []
  for j in df.columns:
    if "ETF" in j:
      l.append(j)
  df.drop(l, axis=1, inplace=True)
  return df.shape
ETF(price)
# (5689, 441)

# Drop 'Error' columns
for i in price.columns:
  if '#ERROR' in i:
    price.drop(i, axis=1, inplace=True)
# (5689, 438)

# Drop VT:FU...   # 14 kí tự
FU = []
for i in price.columns:
  if len(str(price[i][0])) >= 14:
    FU.append(i)
price.drop(FU, axis=1, inplace=True)
# (5689, 435)

# Get column name for price
price_col = []   # 435
x=0
for col in range(0, len(price.columns)):
  price_col.append(str(price.iloc[0,col])[:6])
  x+=1

# Drop 'Code' row and change index name
price.drop("Code", axis=0, inplace =True)
price.index.name = 'Date'

# Deal with dead - delist companies   32 companies + Gia Lai Electricity
price.index = pd.to_datetime(price.index)
def Dead_Delist(df):
  for i in df.columns:
    if 'DEAD - DELIST' in i:
      delist_date = datetime.strptime(i[-8:], "%d/%m/%y")
      df.loc[(delist_date):,i] = np.nan
  return df.shape
Dead_Delist(price)
# (5688, 435)

# Change column names
price.columns = price_col

# Drop companies are established under 5 years
for i in price.columns:
  if price[i].count() < 1260:  # 252days/year x 5years (hàng1 còn tên viết tắt)
    price.drop(i, axis=1, inplace=True)


# Chọn mốc ngày 1/1/2007 - 31/12/2020
price = price.iloc[1826:5480]
# (3654, 337)

# Drop companies don't have BV
price.drop(['VT:C47', 'VT:SMB', 'VT:VCA', '67081W', '67083P'], axis=1, inplace=True)
# (3654, 332)

# Set day 7 of each month for Price
price.index = pd.to_datetime(price.index)
start = datetime(2007, 1, 1)
end = datetime(2020,12,31)
date =[]
timestamp = start
while timestamp < end:
    timestamp += timedelta(days = 1)
    if timestamp.date().strftime('%Y-%m-%d')[-2:] == '07':
        date.append(timestamp.date())

# Drop Saturday and Sunday #nextfill
for i in range(len(date)):
    if date[i].strftime('%A') == "Saturday":
        date[i] -= timedelta(days=1)
    elif date[i].strftime('%A') == "Sunday":
        date[i] -= timedelta(days=2)
    else:
        pass
#print('List 7th day: \n', date)

# DataFrame with only the 7th day
price_month = price.loc[date]
price_month.index.names = ['Day 7 of each month']

#Change full date to date only month and year
price_month.index = pd.to_datetime(price_month.index)
price_month.index = price_month.index.strftime('%Y-%m')
print('Price only month-year: \n', price_month)    # 168 tháng, 332 cty

# %% Marketvalue_day7
mv = pd.read_csv('Marketvalue.csv', low_memory=False, index_col=1, header=1).iloc[:,1:]
mv.index.name = 'Date'

# Change column names
mv_col = []
for i in mv.columns:
  mv_col.append(str(i[:6]))
mv.columns = mv_col

# Synchronize price sheet and marketvalue sheet
mv = mv.loc[:,price.columns]
mv = mv.iloc[1826:5480,:]
# (3654, 340)

# Set day 7 of each month for Marketvalue
mv.index = pd.to_datetime(mv.index)
mv_month = pd.DataFrame(mv, index=date)
mv_month.index.names = ['Day 7 of each month']

#Change full date to date only month and year
mv_month.index = pd.to_datetime(price_month.index)
mv_month.index = mv_month.index.strftime('%Y-%m')

print('Marketvalue only month-year: \n', mv_month)

# %% bv_day7
bv = pd.read_csv('Bookvalue.csv', low_memory=False, index_col=1, header=1).iloc[:,1:]
bv.index.name = 'Day 7 of each month'

# Change column names
bv_col = []
for i in bv.columns:
  bv_col.append(str(i[:6]))
bv.columns = bv_col

# Synchronize 3 sheets
bv = bv.loc[:,price.columns]
bv = bv.iloc[83:252,:]   # 31/1/2007-31/12/2020
# (3654, 332)

#Change full date to date only month and year
bv.index = pd.to_datetime(bv.index)
bv.index = bv.index.strftime('%Y-%m')
month_date = bv.index
bv = bv.iloc[:-1]
bv.index = month_date[1:]
print('Bookvalue only month-year: \n', bv)    # 168 tháng, 340 cty

# Số liệu của BV chính xác, các cty đã huỷ niêm yết sẽ bị NaN
# Số liệu của Price, Marketvalue canh theo giá thị trường ngày cuối cùng giao dịch mà ffill nên k chính xác.

# %% Synchronize 'NaN' in 3 sheets
import math

price_month = price_month.astype('float')
mv_month = mv_month.astype('float')
bv = bv.astype('float')

for i in range(len(bv.columns)):
  for j in range(len(bv.index)):
    if math.isnan(price_month.iloc[j][i]) == True:
      bv.iloc[j][i] = np.nan
      mv_month.iloc[j][i] = np.nan

for i in range(len(bv.columns)):
  for j in range(len(bv.index)):
    if math.isnan(bv.iloc[j][i]) == True:
      price_month.iloc[j][i] = np.nan
      mv_month.iloc[j][i] = np.nan

# %% Price_Rate of return (theo tháng, mốc là ngày 7 mỗi tháng)
rate = price_month.pct_change(periods=1)
rate = rate.iloc[1:,:]   # 2007-02 đến 2020-12
# 7/1/2007-7/12/2020
# (167,332)

mv_month = mv_month.iloc[1:,:]   # 2007-02 đến 2020-12
bv = bv.iloc[1:,:]    # 2007-02 đến 2020-12

# %% Tạo dataframe chứa B/M theo ngày 7 mỗi tháng của các cty
BM = (bv*1000)/mv_month
# (167, 332)

# %% Split companies into 2 dataframes by day 7 of each month
import statistics
import math

r_smb = []
r_hml = []
ri = []

for i in mv_month.index:
  # Mean of each row:
  value = []
  for j in mv_month.columns:
    if math.isnan(mv_month.loc[i,j]) == False:
      value.append(mv_month.loc[i,j])
  meann = statistics.mean(value)
  # Split big-small:
  big = []
  small = []
  df_small = pd.DataFrame()
  df_big = pd.DataFrame()
  for k in mv_month.columns:   # tháng
    if math.isnan(mv_month.loc[i,k]) == False and mv_month.loc[i,k] < meann:
      small.append(k)
    elif math.isnan(mv_month.loc[i,k]) == False and mv_month.loc[i,k] >= meann:
      big.append(k)
  # Create dataframe for big-small:
  df_small = df_small.append(BM.loc[i, small])
  df_big = df_big.append(BM.loc[i, big])
  df_small = np.transpose(df_small)
  df_big = np.transpose(df_big)
  rate_big = rate.loc[i,big].T
  rate_small = rate.loc[i,small].T
  df_big = pd.concat([df_big, rate_big], axis=1)
  df_small = pd.concat([df_small, rate_small], axis=1)
  df_small.columns = ["BM", "Rate"]
  df_big.columns = ["BM", "Rate"]
  # Sort by BM
  df_big = df_big.sort_values(by=['BM'])
  df_small = df_small.sort_values(by=['BM'])
  # 30% BL - 40% BN - 30% BH & 30% SL - 40% SN - 30% SH
  count_big = len(big)   # Số cty lớn
  count_small = len(small)    # Số cty nhỏ
  rbl = df_big.iloc[:int(0.3*count_big),1].mean()
  rbn = df_big.iloc[int(0.3*count_big):int(0.7*count_big),1].mean()
  rbh = df_big.iloc[int(0.7*count_big):,1].mean()
  rsl = df_small.iloc[:int(0.3*count_big),1].mean()
  rsn = df_small.iloc[int(0.3*count_big):int(0.7*count_big),1].mean()
  rsh = df_small.iloc[int(0.7*count_big):,1].mean()
  rsmb = (1/3)*(rsh+rsn+rsl) - (1/3)*(rbh+rbn+rbl)
  rhml = (1/2)*(rbh+rsh) - (1/2)*(rbl+rsl)
  r_smb.append(rsmb)
  r_hml.append(rhml)
  # For model
  m = (rbl+rbn+rbh+rsl+rsn+rsh)/6
  ri.append(m)
# 167 tháng

# Vì r_hml và r_smb bị NaN ở 3 tháng đầu (từ 2007-02 đến 2007-04) nên sẽ drop 3 tháng đó.
r_hml = r_hml[3:]
r_smb = r_smb[3:]
ri = ri[3:]
# 164 tháng từ 2007-05 đến 2020-12

# Build model

# Create dataframe for X, y:

# %% VNINDEX
vnindex = pd.read_csv('/content/capm-data.csv', index_col=0, parse_dates=True, header=0).iloc[1741:5330,0]  # 2/4/2007-31/12/2020
# Series with only the 7th day
vnindex_day7 = vnindex.loc[date[3:]]
vnindex_day7.index.names = ['Year-Month']

# Change full date to date only month and year
vnindex_day7.index = pd.to_datetime(vnindex_day7.index)   # Day 7 of each month
vnindex_day7.index = vnindex_day7.index.strftime('%Y-%m')

# Rate of return by VNINDEX
vnindex_day7.astype('float')
rate_vnindex = vnindex_day7.pct_change()
rate_vnindex.dropna(axis=0, inplace=True)    # 7/5/2007-7/12/2020
rate_vnindex = pd.DataFrame(rate_vnindex, columns=['VNINDEX'])
print('VNINDEX rate only month-year: \n', rate_vnindex)   # 164 tháng

# %% BOND YIELD
bond = pd.read_excel('/content/Bond Yiled.xlsx', index_col=0, parse_dates=True, header=None, skiprows=6).iloc[66:231]
bond.index.names = ['Year-Month']
bond.columns = ['BOND']

# Change full date to date only month and year
bond.index = pd.to_datetime(bond.index)
bond.index = bond.index.strftime('%Y-%m')

bond = bond.fillna(8)  # Fill NaN của bond bằng 8%.
bond = bond/12

bond.astype('float')
rate_bond = bond.pct_change()
rate_bond.dropna(axis=0, inplace=True)    # 7/5/2007-7/12/2020
print('BOND YIELD rate only month-year: \n', rate_bond)    #  164 tháng

# %% Create 'VNINDEX-rf' feature:
r_mkt = []
for i in range(164):
  r_mkt.append(float(rate_vnindex.iloc[i,0]) - float(rate_bond.iloc[i,0]))

# Create dataframe for X:
data_X = pd.DataFrame(np.array(r_smb).T, index=rate_vnindex.index, columns=['RSMB'])
r_hml = pd.DataFrame(np.array(r_hml), index=rate_vnindex.index, columns=['RHML'])
r_mkt = pd.DataFrame(np.array(r_mkt), index=rate_vnindex.index, columns=['RMKT'])
data_X = pd.concat([data_X,r_hml], axis=1)
data_X = pd.concat([data_X,r_mkt], axis=1)

# %% Create dataframe for y:
y = []
for i in range(164):
  y.append(ri[i] - rate_bond.iloc[i,0])
y = pd.DataFrame(y, index=rate_vnindex.index, columns=['y'])
data = pd.concat([data_X, y], axis=1)
# (164,4)

# %% Import library
import statsmodels.formula.api as smf

# y = ri - rf = price_month - rf
# X = smb, hml, rm - rf = VNINDEX - rf

# Run model
model = 'y ~ RMKT + RSMB + RHML'
ff_model = smf.ols(model, data=data).fit()
print(ff_model.summary())
print('hehehe')

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Price only month-year: 
                     VT:VCB    VT:VIC    VT:HPG  ... VT:TIC VT:VTF    67079K
Day 7 of each month                             ...                        
2007-01                NaN       NaN       NaN  ...    NaN    NaN  17575.75
2007-02                NaN       NaN       NaN  ...    NaN    NaN   21212.1
2007-03                NaN       NaN       NaN  ...    NaN    NaN  26666.64
2007-04                NaN       NaN       NaN  ...    NaN    NaN  21515.14
2007-05                NaN       NaN       NaN  ...    NaN    NaN  19696.95
...                    ...       ...       ...  ...    ...    ...       ...
2020-08              82900  77777.75  17925.92  ...    NaN    NaN       NaN
2020-09              83600  81333.31  18148.14  ...    NaN    NaN       NaN
2020-10              85000  82222.19  20592.59  ...    NaN    NaN       NaN
2020-11              85000  93422.19  22518.51  ...    NaN    NaN       NaN
2020-12              93100  94044.44  28296.29  ...    NaN    N